In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas
#os.environ['THEANO_FLAGS'] = 'optimizer=None'

caltech_dir = "./images/"
categories = ["Beagle","bulldog","Chihuahua","corgi","Greyhound","Husky","Maltese",
              "Pomeranian","Shepherd","Shiba","ShihTzu","Toypoodle","YorshireTerrier"]
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    img_num = 0
    
    print(cat, " 파일 길이 : ", len(files))
    
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img) / 255.0

        X.append(data)
        y.append(label)

        if i % 3000 == 0:
            print(cat, " : ", f)
        img_num += 1
        
        if i >= 6000 : break

    print (cat, "이미지 개수 ", img_num)
    
print ("전체 이미지 개수 : ", len(X))

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식

X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./multi_image_data.npy", xy)

print("ok", len(y))

Beagle  파일 길이 :  2101
Beagle  :  ./images//Beagle\adorable beagle1.jpg
Beagle 이미지 개수  2101
bulldog  파일 길이 :  1822
bulldog  :  ./images//bulldog\adorable bulldog1.jpg
bulldog 이미지 개수  1822
Chihuahua  파일 길이 :  1905
Chihuahua  :  ./images//Chihuahua\adorable Chihuahua1.jpg
Chihuahua 이미지 개수  1905
corgi  파일 길이 :  1747
corgi  :  ./images//corgi\adorable corgi1.jpg
corgi 이미지 개수  1747
Greyhound  파일 길이 :  1834
Greyhound  :  ./images//Greyhound\adorable greyhound1.jpg
Greyhound 이미지 개수  1834
Husky  파일 길이 :  2044
Husky  :  ./images//Husky\adorable husky10.jpg
Husky 이미지 개수  2044
Maltese  파일 길이 :  1983
Maltese  :  ./images//Maltese\baby maltese1.jpg
Maltese 이미지 개수  1983
Pomeranian  파일 길이 :  1918
Pomeranian  :  ./images//Pomeranian\black pomeranian1.jpg
Pomeranian 이미지 개수  1918
Shepherd  파일 길이 :  1745
Shepherd  :  ./images//Shepherd\adorable shepherd1.jpg
Shepherd 이미지 개수  1745
Shiba  파일 길이 :  1453
Shiba  :  ./images//Shiba\adorable shiba1.jpg
Shiba 이미지 개수  1453
ShihTzu  파일 길이 :  2029
ShihTzu  :  ./imag

C:\Users\YUBIN\anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


ok 24420


In [2]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

X_train, X_test, y_train, y_test = np.load('./multi_image_data.npy',allow_pickle = True)
print(X_train.shape)
print(X_train.shape[0])

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\YUBIN\anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


(18315, 64, 64, 3)
18315


In [3]:
#정규화 일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [4]:
with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
    model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_dir = './model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    model_path = model_dir + '/multi_img_classification.model'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
__________

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Train on 18315 samples, validate on 6105 samples
Epoch 1/50
10880/18315 [================>.............] - ETA: 13:00:03 - loss: 2.5628 - acc: 0.0876

In [ ]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_oss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()